In [1]:
import matplotlib.pyplot as plt
import numpy as np

from importlib import reload
from matplotlib import rc
from neurobiases import (TriangularModel,
                         EMSolver,
                         solver_utils)
%matplotlib inline

In [2]:
tm = TriangularModel(
    model='linear',
    parameter_design='direct_response',
    M=10,
    N=10,
    K=1,
    corr_cluster=0.25,
    corr_back=0.1,
    coupling_distribution='gaussian',
    coupling_sparsity=0.5,
    coupling_loc=0.,
    coupling_scale=0.25,
    coupling_rng=2332,
    tuning_distribution='gaussian',
    tuning_sparsity=0.5,
    tuning_loc=0.,
    tuning_scale=0.25,
    tuning_rng=23456542,
    stim_distribution='uniform'
)

In [3]:
X, Y, y = tm.generate_samples(n_samples=1000, rng=2332)

In [4]:
solver = EMSolver(
    X, Y, y, K=1,
    solver='ow_lbfgs',
    max_iter=20,
    tol=0,
    c_coupling=1e-2,
    c_tuning=1e-2
).fit_em(verbose=True, store_parameters=True)

Initial marginal likelihood: 1356.116774
Iteration 1, del=15.730496193, mll=22688.506528
Iteration 2, del=0.002562105, mll=22746.636872
Iteration 3, del=0.011447671, mll=23007.032877
Iteration 4, del=0.009981463, mll=23236.676726
Iteration 5, del=0.002406513, mll=23292.596092
Iteration 6, del=0.000603415, mll=23306.651199
Iteration 7, del=0.000190792, mll=23311.097910
Iteration 8, del=0.000071867, mll=23312.773211
Iteration 9, del=0.000026801, mll=23313.398021
Iteration 10, del=0.000008470, mll=23313.595493
Iteration 11, del=0.000001828, mll=23313.638115
Iteration 12, del=0.000002159, mll=23313.688459
Iteration 13, del=0.000000360, mll=23313.696860
Iteration 14, del=0.000000692, mll=23313.713000
Iteration 15, del=0.000000860, mll=23313.733046
Iteration 16, del=0.000001373, mll=23313.701044
Iteration 17, del=0.000000621, mll=23313.686575
Iteration 18, del=0.000000187, mll=23313.690943
Iteration 19, del=0.000001577, mll=23313.654188
Iteration 20, del=0.000000147, mll=23313.657623
